In [2]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# bilud the vocabulary of char and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [120]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [121]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [122]:
C = torch.randn((27,2))

In [123]:
# directly indexing
C[5]

tensor([-0.1347,  0.7581])

In [124]:
# indexing with list
C[[5,6,7]]

tensor([[-0.1347,  0.7581],
        [ 0.6693,  0.8802],
        [-0.7145, -1.0317]])

In [125]:
# this is a alternative to indexing using one hot function
# one hot function takes torch.tensor as a input
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.1347,  0.7581])

In [126]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [127]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [128]:
# concatinatesd the 32,3,2 --> 32,6
torch.cat([emb[:, 0,:],emb[:, 1,:],emb[:, 2,:]], 1).shape

torch.Size([228146, 6])

In [129]:
# generelised
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([228146, 6])

In [130]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [40]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [44]:
# other way to get the same result
emb.view(emb.shape(0),6) == torch.cat(torch.unbind(emb,1),1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [55]:
# here emb.shape(0) and -1 will automatically take it as the no.(32) 
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [56]:
h

tensor([[ 1.0000,  0.9589, -0.7869,  ...,  0.9975,  0.9921, -0.9862],
        [ 0.9604,  0.9932,  0.9969,  ...,  0.9922,  0.0229, -0.9824],
        [-0.6567,  0.9995,  0.9986,  ..., -0.7138, -0.9980,  0.4930],
        ...,
        [ 1.0000,  0.9508,  0.7536,  ...,  0.9856,  0.9998, -0.5988],
        [-0.9978,  0.9984,  0.9067,  ...,  0.9996, -0.9979,  0.9829],
        [ 1.0000,  0.9999,  0.9813,  ..., -0.9861, -0.7462,  0.9984]])

In [52]:
h.shape

torch.Size([32, 100])

In [57]:
(emb.view(-1,6) @ W1).shape

torch.Size([32, 100])

In [58]:
b1.shape

torch.Size([100])

In [59]:
# Broadcasting
# 32, 100
#  1, 100

In [60]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [61]:
logits = h @ W2 + b2

In [62]:
logits.shape

torch.Size([32, 27])

In [64]:
counts =logits.exp()

In [67]:
prob = counts / counts.sum(1, keepdims=True)

In [68]:
prob.shape

torch.Size([32, 27])

In [81]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.0820)

In [73]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [82]:
# ------------ now made respectable :) ---------------

In [185]:
X.shape, Y.shape # dataset

(torch.Size([228146, 3]), torch.Size([228146]))

In [211]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn((100), generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn((27), generator=g)
parameters = [C,W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [212]:
 sum(p.nelement() for p in parameters)

3481

In [213]:
# torch.linespace(0.001,1,1000) -->
lre = torch.linspace(-3,0,1000)
lrs = 10**lre


In [221]:
lri = []
lossi = []
for i in range(10000):

    # minibatch for faster and efficeient less reliable as 
    # based on samll part of data
    ix = torch.randint(0,X.shape[0],(32,))
    
    # forward pass
    emb = C[X[ix]] # (32, 3, 2)
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    #counts =logits.exp()
    #prob = counts / counts.sum(1, keepdims=True)
    #loss = -prob[torch.arange(32), Y].log().mean()
    loss = F.cross_entropy(logits,Y[ix]) 
    #print(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    #lr = lrs[i]
    lr = 0.1
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    #lri.append(lre[i])
    #lossi.append(loss.item())
    
#print(loss.item())

In [222]:
emb = C[X] # (32, 3, 2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Y)
loss

tensor(2.4638, grad_fn=<NllLossBackward0>)

In [1]:
# training split, dev/validation split, test split
# 80% , 10%,  10%